# **1. Import necess lib and Create Spark Session** #

In [1]:
# Import pyspark, SparkSession
import pyspark
from pyspark.sql import SparkSession

# Import SparkFunction
import pyspark.sql.functions as F
from pyspark.sql.types import *

StatementMeta(, e426c28c-019d-448f-8cd3-c173a06af347, 3, Finished, Available)

In [2]:
# Create Spark Session
spark = SparkSession.builder.appName("ETL_Boston_Crime") \
                            .config('spark.cores.max', "16") \
                            .config("spark.executor.memory", "70g") \
                            .config("spark.driver.memory", "50g") \
                            .config("spark.memory.offHeap.enabled",True) \
                            .config("spark.memory.offHeap.size","16g") \
                            .getOrCreate()
    
# Print Spark version
print("Spark version: ", spark.version)

StatementMeta(, e426c28c-019d-448f-8cd3-c173a06af347, 4, Finished, Available)

Spark version:  3.4.1.5.3-117503204


# **2. Extract data** #

In [3]:
!pip install kaggle

StatementMeta(, e426c28c-019d-448f-8cd3-c173a06af347, 5, Finished, Available)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.6/84.6 kB 7.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - \ done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.2/78.2 kB 41.9 MB/s eta 0:00:00
  Created wheel for kaggle: filename=kaggle-1.6.6-py3-none-any.whl size=111943 sha256=f4b57fd84e9d3088dbd466fba0ab9fe312c850e44593b70fc395517d3e5a53d7
  Stored in directory: /home/trusted-service-user/.cache/pip/wheels/53/34/8c/8ca3450d17206d9e37e1ee3aeb47cbb2873d22a9e0c60eb137
Successfully built kaggle


In [4]:
# Define path download file and kaggle_user
import os
os.chdir("/lakehouse/default/Files")
os.environ['KAGGLE_USERNAME'] = 'phatnguyen2609'
os.environ['KAGGLE_KEY'] = '430723973472d10a03fa95b817a03ce7'

# Import Kaggle API and download file
from kaggle.api.kaggle_api_extended import KaggleApi
api = KaggleApi()
api.authenticate()
api.dataset_download_file('AnalyzeBoston/crimes-in-boston/data?select=crime.csv', 'crime.csv')
api.dataset_download_file('AnalyzeBoston/crimes-in-boston/data?select=offense_codes.csv', 'offense_codes.csv')

StatementMeta(, e426c28c-019d-448f-8cd3-c173a06af347, 6, Finished, Available)

True

In [5]:
# Data file_path of crime.csv.zip
file_zip = "/lakehouse/default/Files/crime.csv.zip"

# Unzip data
import zipfile

with zipfile.ZipFile(file_zip, 'r') as zip_ref:
    zip_ref.extractall('/lakehouse/default/Files')

StatementMeta(, e426c28c-019d-448f-8cd3-c173a06af347, 7, Finished, Available)

In [6]:
# Define file_path
file_path = "Files/crime.csv"

# Read dataset crime.csv into df_boston
df_boston = spark.read.format("csv") \
                      .option("header", True) \
                      .option("inferSchema", True) \
                      .load(file_path)

# Show df_boston
df_boston.show(5)

StatementMeta(, e426c28c-019d-448f-8cd3-c173a06af347, 8, Finished, Available)

+---------------+------------+--------------------+--------------------+--------+--------------+--------+-------------------+----+-----+-----------+----+----------+-----------+-----------+------------+--------------------+
|INCIDENT_NUMBER|OFFENSE_CODE|  OFFENSE_CODE_GROUP| OFFENSE_DESCRIPTION|DISTRICT|REPORTING_AREA|SHOOTING|   OCCURRED_ON_DATE|YEAR|MONTH|DAY_OF_WEEK|HOUR|  UCR_PART|     STREET|        Lat|        Long|            Location|
+---------------+------------+--------------------+--------------------+--------+--------------+--------+-------------------+----+-----+-----------+----+----------+-----------+-----------+------------+--------------------+
|     I182070945|         619|             Larceny|  LARCENY ALL OTHERS|     D14|           808|    null|2018-09-02 13:00:00|2018|    9|     Sunday|  13|  Part One| LINCOLN ST|42.35779134|-71.13937053|(42.35779134, -71...|
|     I182070943|        1402|           Vandalism|           VANDALISM|     C11|           347|    null|201

In [7]:
# Define file_path
file_path = "Files/offense_codes.csv"

# Read dataset offense_codes.csv into dim_offensecode_df
dim_offensecode_df = spark.read.format("csv") \
                               .option("header", True) \
                               .option("inferSchema", True) \
                               .load(file_path)

# Show dataframe
dim_offensecode_df.show(5)

StatementMeta(, e426c28c-019d-448f-8cd3-c173a06af347, 9, Finished, Available)

+----+--------------------+
|CODE|                NAME|
+----+--------------------+
| 612|LARCENY PURSE SNA...|
| 613| LARCENY SHOPLIFTING|
| 615|LARCENY THEFT OF ...|
|1731|              INCEST|
|3111|LICENSE PREMISE V...|
+----+--------------------+
only showing top 5 rows



# **3. Transformation data** #

## 3.1. Transform dim_offensecode_df ##

In [8]:
# Check duplicates of dim_offensecode_df
dim_offensecode_df.exceptAll(dim_offensecode_df.dropDuplicates(['CODE'])).show()

StatementMeta(, e426c28c-019d-448f-8cd3-c173a06af347, 10, Finished, Available)

+----+--------------------+
|CODE|                NAME|
+----+--------------------+
| 311|ROBBERY - KNIFE -...|
|2642|             TRUANCY|
|1847|DRUGS - POSS CLAS...|
| 727|AUTO THEFT LEASE/...|
|2005|  CHILD ENDANGERMENT|
| 413| ASSAULT D/W - OTHER|
|3106|PROPERTY - ACCIDE...|
|2407|ANNOYING AND ACCO...|
|1502|FIREARM/WEAPON - ...|
|3620|REPORT AFFECTING ...|
| 724|          AUTO THEFT|
|2670|CRIMINAL HARRASSMENT|
|2660|       OTHER OFFENSE|
| 611|LARCENY PICK-POCK...|
|2907|M/V - LEAVING SCE...|
| 619|LARCENY OTHER $20...|
| 541|B&E NON-RESIDENCE...|
| 351|ROBBERY ATTEMPT -...|
| 540|B&E NON-RESIDENCE...|
|3110|SERVICE TO OTHER ...|
+----+--------------------+
only showing top 20 rows



In [9]:
print("Number of rows before drop duplicates dim_offensecode_df: ", 
                                                        dim_offensecode_df.count())

# Drop duplicates
dim_offensecode_df = dim_offensecode_df.dropDuplicates()

print("Number of rows after drop duplicates dim_offensecode_df: ", 
                                                        dim_offensecode_df.count())

StatementMeta(, e426c28c-019d-448f-8cd3-c173a06af347, 11, Finished, Available)

Number of rows before drop duplicates dim_offensecode_df:  576
Number of rows after drop duplicates dim_offensecode_df:  493


## 3.2. Transform df_boston ##

In [10]:
# Describe df_boston
df_boston.describe().show()

StatementMeta(, e426c28c-019d-448f-8cd3-c173a06af347, 12, Finished, Available)

+-------+---------------+-----------------+------------------+--------------------+--------+------------------+--------+------------------+------------------+-----------+------------------+--------+-----------+-----------------+------------------+--------------------+
|summary|INCIDENT_NUMBER|     OFFENSE_CODE|OFFENSE_CODE_GROUP| OFFENSE_DESCRIPTION|DISTRICT|    REPORTING_AREA|SHOOTING|              YEAR|             MONTH|DAY_OF_WEEK|              HOUR|UCR_PART|     STREET|              Lat|              Long|            Location|
+-------+---------------+-----------------+------------------+--------------------+--------+------------------+--------+------------------+------------------+-----------+------------------+--------+-----------+-----------------+------------------+--------------------+
|  count|         319073|           319073|            319073|              319073|  317308|            319073|    1019|            319073|            319073|     319073|            319073|  31

In [11]:
# Value count SHOOTING column for check error data
df_boston.groupBy("SHOOTING").count().show()

StatementMeta(, e426c28c-019d-448f-8cd3-c173a06af347, 13, Finished, Available)

+--------+------+
|SHOOTING| count|
+--------+------+
|    null|318054|
|       Y|  1019|
+--------+------+



In [12]:
# Value count for Lat, Long for check error data ('-1' is error value)
df_boston.groupBy("Lat").count().orderBy("count", ascending = False).show()

StatementMeta(, e426c28c-019d-448f-8cd3-c173a06af347, 14, Finished, Available)

+-----------+-----+
|        Lat|count|
+-----------+-----+
|       null|19999|
|42.34862382| 1243|
|42.36183857| 1208|
|42.28482577| 1121|
|42.32866284| 1042|
|42.25621592|  898|
|42.29755533|  783|
|42.34128751|  773|
|       -1.0|  745|
|42.33152148|  735|
| 42.3523119|  688|
|42.33954199|  655|
|42.32696647|  652|
|42.35512339|  584|
|42.30971857|  573|
|42.29848866|  562|
|42.33401829|  561|
|42.33367922|  550|
|42.35095909|  523|
|  42.310434|  523|
+-----------+-----+
only showing top 20 rows



In [13]:
# Value count for Long column ('-1' is error value)
df_boston.groupBy("Long").count().orderBy("count", ascending = False).show()

StatementMeta(, e426c28c-019d-448f-8cd3-c173a06af347, 15, Finished, Available)

+------------+-----+
|        Long|count|
+------------+-----+
|        null|19999|
|-71.08277637| 1243|
|-71.05976489| 1208|
|-71.09137369| 1121|
|-71.08563401| 1042|
|-71.12401947|  898|
| -71.0597091|  783|
|-71.05467933|  773|
|        -1.0|  745|
|-71.07085307|  735|
| -71.0637051|  688|
|-71.06940877|  655|
|-71.06198607|  652|
| -71.0608798|  584|
|-71.10429432|  573|
|-71.06313294|  562|
|-71.07638124|  561|
|-71.09187755|  550|
| -71.0741278|  523|
| -71.0613401|  523|
+------------+-----+
only showing top 20 rows



In [14]:
# Value count for UCR_Part ('Other' is a error value)
df_boston.groupBy("UCR_PART").count().show()

StatementMeta(, e426c28c-019d-448f-8cd3-c173a06af347, 16, Finished, Available)

+----------+------+
|  UCR_PART| count|
+----------+------+
|      null|    90|
|     Other|  1232|
|  Part Two| 97569|
|Part Three|158553|
|  Part One| 61629|
+----------+------+



In [15]:
# Replace value for SHOOTING column
df_boston = df_boston.withColumn("SHOOTING", F.when(F.col('SHOOTING') == 'Y', "Yes")
                                              .otherwise("No"))

# Drop value '-1' from Lat, Long
df_boston = df_boston.filter(F.col('Lat') != '-1')
df_boston = df_boston.filter(F.col('Long') != '-1')

# Drop value 'Other' from UCR_PART
df_boston = df_boston.filter(F.col('UCR_PART') != 'Other')

StatementMeta(, e426c28c-019d-448f-8cd3-c173a06af347, 17, Finished, Available)

In [16]:
# Count before drop null value
print("Before drop null values: ", df_boston.count())

# Drop null value
df_boston = df_boston.dropna()

# Count after drop null value
print("After drop null values: ", df_boston.count())

StatementMeta(, e426c28c-019d-448f-8cd3-c173a06af347, 18, Finished, Available)

Before drop null values:  297088
After drop null values:  295277


# 4. Load to DataWarehouse (Lakehouse) #

In [24]:
# Save file for Machine Learning
df_boston.write.csv("Files/boston_crime.csv")

StatementMeta(, e426c28c-019d-448f-8cd3-c173a06af347, 26, Finished, Available)

In [18]:
# Make all column CAP
df_boston = df_boston.withColumnRenamed("Lat", "LAT")
df_boston = df_boston.withColumnRenamed("Long", "LONG")
df_boston = df_boston.withColumnRenamed("Location", "LOCATION")

StatementMeta(, e426c28c-019d-448f-8cd3-c173a06af347, 20, Finished, Available)

In [19]:
# Function create dim_table
def create_dim_table(df_base, column_from_base, name_id_column, name_temp_view, query):
    # Choose column
    dim_table_new = df_base[column_from_base]

    # Drop duplicates data
    dim_table_new = dim_table_new.dropDuplicates()

    # Create column ID
    dim_table_new = dim_table_new.withColumn(name_id_column, 
                                             F.monotonically_increasing_id() + 1)
    
    # Change order of column
        # Create tempview
    dim_table_new.createOrReplaceTempView(name_temp_view)
        # Query
    dim_table_new = spark.sql(query)
        # Drop tempview
    spark.catalog.dropTempView(name_temp_view)

    # Return result
    return dim_table_new

StatementMeta(, e426c28c-019d-448f-8cd3-c173a06af347, 21, Finished, Available)

## 4.1. Create Dim_DateTime ##

Schema: OCCURRED_ON_DATE, DATE, MONTH, QUARTER, YEAR, DAY_OF_WEEK, HOUR

In [20]:
df_base = df_boston
column_from_base = ['OCCURRED_ON_DATE', 'YEAR', 'MONTH', 'DAY_OF_WEEK', 'HOUR']
name_id_column = "DATETIME_ID"
name_temp_view = "dim_time"
query = ''' 
        SELECT DATETIME_ID, OCCURRED_ON_DATE, MONTH, YEAR, DAY_OF_WEEK, HOUR
        FROM dim_time
        '''

# Use function
dim_datetime_df = create_dim_table(df_base, column_from_base, name_id_column, 
                                   name_temp_view, query)

# Create newcolumn
dim_datetime_df = dim_datetime_df.withColumn('DATE', 
                                              F.to_date(F.col('OCCURRED_ON_DATE')))

# Show dataframe
dim_datetime_df.show(5)

StatementMeta(, e426c28c-019d-448f-8cd3-c173a06af347, 22, Finished, Available)

+-----------+-------------------+-----+----+-----------+----+----------+
|DATETIME_ID|   OCCURRED_ON_DATE|MONTH|YEAR|DAY_OF_WEEK|HOUR|      DATE|
+-----------+-------------------+-----+----+-----------+----+----------+
|          1|2018-09-01 04:47:00|    9|2018|   Saturday|   4|2018-09-01|
|          2|2018-09-01 00:06:00|    9|2018|   Saturday|   0|2018-09-01|
|          3|2018-08-31 13:16:00|    8|2018|     Friday|  13|2018-08-31|
|          4|2018-08-31 10:12:00|    8|2018|     Friday|  10|2018-08-31|
|          5|2018-08-30 11:46:00|    8|2018|   Thursday|  11|2018-08-30|
+-----------+-------------------+-----+----+-----------+----+----------+
only showing top 5 rows



## 4.2. Create Dim_Location ##

In [21]:
df_base = df_boston
column_from_base = ['STREET', 'LAT', 'LONG', 'LOCATION']
name_id_column = "LOCATION_ID"
name_temp_view = "dim_location"
query = ''' 
        SELECT LOCATION_ID, STREET, LAT, LONG, LOCATION
        FROM dim_location
        '''

# Use function
dim_location_df = create_dim_table(df_base, column_from_base, name_id_column, 
                                   name_temp_view, query)

# Show dataframe
dim_location_df.show(5)

StatementMeta(, e426c28c-019d-448f-8cd3-c173a06af347, 23, Finished, Available)

+-----------+-------------+-----------+------------+--------------------+
|LOCATION_ID|       STREET|        LAT|        LONG|            LOCATION|
+-----------+-------------+-----------+------------+--------------------+
|          1|WASHINGTON ST|42.35042609|-71.16637332|(42.35042609, -71...|
|          2|GALLIVAN BLVD|42.28158633|-71.06033424|(42.28158633, -71...|
|          3|     SAVIN ST|42.31719004|-71.08167267|(42.31719004, -71...|
|          4| CLARENDON ST|42.34854636|-71.07416609|(42.34854636, -71...|
|          5|    SUMMER ST| 42.3465477|-71.04284093|(42.34654770, -71...|
+-----------+-------------+-----------+------------+--------------------+
only showing top 5 rows



## 4.3. Create Dim_Ucr_Part ##

In [23]:
df_base = df_boston
column_from_base = ['UCR_PART']
name_id_column = "UCR_PART_ID"
name_temp_view = "dim_ucrpart"
query = ''' 
        SELECT UCR_PART_ID, UCR_PART
        FROM dim_ucrpart
        '''

# Use function
dim_ucrpart_df = create_dim_table(df_base, column_from_base, name_id_column, 
                                   name_temp_view, query)

# Show dataframe
dim_ucrpart_df.show(5)

StatementMeta(, e426c28c-019d-448f-8cd3-c173a06af347, 25, Finished, Available)

+-----------+----------+
|UCR_PART_ID|  UCR_PART|
+-----------+----------+
|          1|  Part Two|
|          2|Part Three|
|          3|  Part One|
+-----------+----------+



## 4.4. Create Dim_OffenseCode ##

In [27]:
dim_offensecode_df = dim_offensecode_df.withColumnRenamed("CODE", "OFFENSE_CODE")
dim_offensecode_df.printSchema()

StatementMeta(, e426c28c-019d-448f-8cd3-c173a06af347, 29, Finished, Available)

root
 |-- OFFENSE_CODE: integer (nullable = true)
 |-- NAME: string (nullable = true)



# 4.5. Create Fact table #

In [41]:
# Join Dataframe and Choose column replace with ID column
fact_df = df_boston.join(dim_datetime_df, on = ['OCCURRED_ON_DATE', 'YEAR', 'MONTH', 'DAY_OF_WEEK', 'HOUR'], how = "inner") \
                   .join(dim_location_df, on = ["STREET", "LAT", "LONG", "LOCATION"], how = "inner") \
                   .join(dim_ucrpart_df, on = "UCR_PART", how = "inner") \
                   .select(["INCIDENT_NUMBER", "DATETIME_ID", "LOCATION_ID", 
                            "OFFENSE_CODE", "UCR_PART_ID", "OFFENSE_CODE_GROUP",
                            "OFFENSE_DESCRIPTION", "DISTRICT", "REPORTING_AREA",
                            "SHOOTING"])

StatementMeta(, e426c28c-019d-448f-8cd3-c173a06af347, 43, Finished, Available)

In [43]:
fact_df.printSchema()

StatementMeta(, e426c28c-019d-448f-8cd3-c173a06af347, 45, Finished, Available)

root
 |-- INCIDENT_NUMBER: string (nullable = true)
 |-- DATETIME_ID: long (nullable = false)
 |-- LOCATION_ID: long (nullable = false)
 |-- OFFENSE_CODE: integer (nullable = true)
 |-- UCR_PART_ID: long (nullable = false)
 |-- OFFENSE_CODE_GROUP: string (nullable = true)
 |-- OFFENSE_DESCRIPTION: string (nullable = true)
 |-- DISTRICT: string (nullable = true)
 |-- REPORTING_AREA: string (nullable = true)
 |-- SHOOTING: string (nullable = false)



# 5. Save to Lakehouse #

In [53]:
# Sort Dataframe for better view
fact_df = fact_df.orderBy(["DATETIME_ID", "LOCATION_ID", "UCR_PART_ID"], ascending = True)
dim_datetime_df = dim_datetime_df.orderBy("DATETIME_ID", ascending = True)
dim_location_df = dim_location_df.orderBy("LOCATION_ID", ascending = True)
dim_ucrpart_df = dim_ucrpart_df.orderBy("UCR_PART_ID", ascending = True)

StatementMeta(, e426c28c-019d-448f-8cd3-c173a06af347, 55, Finished, Available)

In [51]:
# Save dataframe to LakeHouse to Create DataWarehouse into LakeHouse
def save_df_table(table_name, df_save, format_save = "delta"):
    # Define delta_path
    delta_path = f"Tables/{table_name}"

    # Save as table
    df_save.write.format(format_save).mode("overwrite").save(delta_path)

StatementMeta(, e426c28c-019d-448f-8cd3-c173a06af347, 53, Finished, Available)

In [52]:
# Crete Dimension tables
save_df_table("Dim_DateTime", dim_datetime_df)
save_df_table("Dim_Location", dim_location_df)
save_df_table("Dim_UcrPart", dim_ucrpart_df)
save_df_table("Dim_OffenseCode", dim_offensecode_df)

# Create Fact Table
save_df_table("Fact_BostonCrime", fact_df)

StatementMeta(, e426c28c-019d-448f-8cd3-c173a06af347, 54, Finished, Available)